In [1]:
import requests
import config
import json
import base64


RIOT_BASE_URL = "https://europe.api.riotgames.com/riot/account/v1/accounts/by-riot-id/"

# DO NOT COMMIT THIS TO GITHUB IDIOT
RIOT_API_KEY = config.API_KEY


class Client():

    def __init__(self, username, tag):
        self.username = username
        self.tag = tag

    
    def get_puuid(self):
        response = requests.get(f"{RIOT_BASE_URL}{self.username}/{self.tag}?api_key={RIOT_API_KEY}")
        puuid = response.json()['puuid']
        return puuid
    




In [ ]:
client = Client('KingRobbolo', 'EUW')
player_id = client.get_puuid()
print(player_id)


match_response = requests.get(f"https://euw1.api.riotgames.com/lol/spectator/v5/active-games/by-summoner/{player_id}?api_key={RIOT_API_KEY}")

full_response = json.loads(match_response.text)

print(full_response)


In [ ]:
import subprocess
import requests
import json

def get_lcu_credentials():
    try:
        # Run command to get LCU process details
        result = subprocess.check_output('wmic process where "name=\'LeagueClientUx.exe\'" get CommandLine', shell=True).decode()
        lines = result.split("\n")
        
        port, token = None, None
        for line in lines:
            if "--app-port=" in line:
                port = line.split("--app-port=")[1].split()[0][:-1]
            if "--remoting-auth-token=" in line:
                token = line.split("--remoting-auth-token=")[1].split()[0][:-1]
                print(port,token)
        return port, token
    
    except Exception as e:
        print("Error getting LCU credentials:", e)
        return None, None

def get_aram_lobby_data(port, token):
    url = f"https://127.0.0.1:{port}/lol-champ-select/v1/session"
    auth_str = f"riot:{token}"
    encoded_auth = base64.b64encode(auth_str.encode()).decode()

    headers = {
        "Authorization": f"Basic {encoded_auth}"
    }
    print(url)
    
    try:
        response = requests.get(url, headers=headers, verify=False)
        if response.status_code == 200:
            return response.json()
        else:
            print("Error:", response.status_code, response.text)
            return None
    except Exception as e:
        print("Request failed:", e)
        return None

# Get LCU credentials
port, token = get_lcu_credentials()
if port and token:
    print(f"LCU API running on port {port}")
    lobby_data = get_aram_lobby_data(port, token)
    with open ("lobby_dump.json", "w") as file:
        json.dump(lobby_data, file, indent=4)
else:
    print("League Client not running or credentials not found.")




In [ ]:
# store local lobby stuff
# extract benched champs ids
# champid to champ lookup / db
# front end magic
